In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We define our model with the MobileNetV2 model that is defined in the Tensorflow library. I used the article linked below to help me.

We then compile the model with the correct accuracy and loss function.

In [2]:
# Inspired by https://medium.com/hackernoon/tf-serving-keras-mobilenetv2-632b8d92983c
# and https://github.com/malnakli/ML/blob/master/tf_serving_keras_mobilenetv2/main.ipynb
import gc
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
   

target_size = 60

def build_model( ):
    input_tensor = Input(shape=(target_size, target_size, 3))
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=(target_size, target_size, 3),
        pooling='avg')
    
    for layer in base_model.layers:
        layer.trainable = True  # trainable has to be false in order to freeze the layers
            
    op = Dense(256, activation='relu')(base_model.output)
    op = Dropout(.25)(op)
        
        ##
        # softmax: calculates a probability for every possible class.
        #
        # activation='softmax': return the highest probability;
        # for example, if 'Coat' is the highest probability then the result would be 
        # something like [0,0,0,0,1,0,0,0,0,0] with 1 in index 5 indicate 'Coat' in our case.
        ##
    output_tensor = Dense(10, activation='softmax')(op)
    
    model = Model(inputs=input_tensor, outputs=output_tensor)
    
    
    return model
    
    
from tensorflow.keras.optimizers import Adam


model = build_model()
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])   

9412608/9406464 [==============================] - 0s 0us/step


We load the fashion MNIST data set.

In [3]:
batchSize = 64

# Load the fashion-mnist train data and test data
(x_train, y_train), (x_test, y_test) = load_data()
    
    
    
norm_x_train = x_train.astype('float32') / 255
norm_x_test = x_test.astype('float32') / 255

from tensorflow.keras.utils import to_categorical
   
encoded_y_train = to_categorical(y_train, num_classes=10, dtype='float32')
encoded_y_test = to_categorical(y_test, num_classes=10, dtype='float32')
   
del x_train
del x_test
del y_train
del y_test
gc.collect()
    

4423680/4422102 [==============================] - 0s 0us/step


44

Here is the function used on the data set to make it a correct input for our model.

In [4]:
from sklearn.utils import shuffle
from tensorflow.image import resize, grayscale_to_rgb
    
    
def preprocess_images(x):

        
    first = True
    data= []
      
    batch = np.asarray(x)
    batch = batch[..., np.newaxis]
    batch = resize(batch, (target_size, target_size))
           
    batch = grayscale_to_rgb(tf.convert_to_tensor(batch))
    batch = np.asarray(batch)
           
    return (tf.convert_to_tensor(batch.astype(np.float32)))


We apply the function on the training data set and we train our model.

In [5]:
norm_x_train = preprocess_images(norm_x_train)

    
    
train_dataset = tf.data.Dataset.from_tensor_slices((norm_x_train, encoded_y_train)).shuffle(norm_x_train.shape[0], reshuffle_each_iteration=True).batch(batchSize)

    
del norm_x_train
del encoded_y_train
gc.collect()

    
model.fit(
        train_dataset,
        batch_size = batchSize,
        steps_per_epoch=100,
        verbose=1,
        epochs=5)
    

del train_dataset
gc.collect()


Epoch 1/5
100/100 [==============================] - 15s 34ms/step - loss: 1.1111 - categorical_accuracy: 0.6473
Epoch 2/5
100/100 [==============================] - 4s 36ms/step - loss: 0.5076 - categorical_accuracy: 0.8364
Epoch 3/5
100/100 [==============================] - 3s 31ms/step - loss: 0.3777 - categorical_accuracy: 0.8675
Epoch 4/5
100/100 [==============================] - 3s 33ms/step - loss: 0.4014 - categorical_accuracy: 0.8723
Epoch 5/5
100/100 [==============================] - 3s 31ms/step - loss: 0.3542 - categorical_accuracy: 0.8779


8

We apply the function on our test set and we evaluate the model.

In [6]:
norm_x_test = preprocess_images(norm_x_test)

test_dataset = tf.data.Dataset.from_tensor_slices((norm_x_test, encoded_y_test)).shuffle(norm_x_test.shape[0], reshuffle_each_iteration=True).batch(batchSize)
del norm_x_test
gc.collect()
    

model.evaluate(
        test_dataset,
        steps=100,
        verbose=1)

100/100 [==============================] - 3s 11ms/step - loss: 0.9976 - categorical_accuracy: 0.7922


[0.9975534677505493, 0.792187511920929]